In [1]:
import pickle
# add path
import sys
import torch

sys.path.append(r"C:\Users\goelm\OneDrive\Desktop\mawps")

path = r"C:\Users\goelm\OneDrive\Desktop\mawps"

dataloader = pickle.load(open(path + '\mawps-dataloader.pkl', 'rb'))
# model = pickle.load(open(path + '\mawps-model.pkl', 'rb'))
dataset = pickle.load(open(path + '\mawps-dataset.pkl', 'rb'))
trainer = pickle.load(open(path + '\mawps-model-trainer.pkl', 'rb'))

In [2]:
# model = torch.load(r"C:\Users\goelm\OneDrive\Desktop\model.pth", map_location=torch.device('cuda'))
# model

In [2]:
import pandas as pd
from backend.comparator_tree import *

df = pd.read_pickle('backend/data/TextData/all_df.pkl')

tree = df['tree']

In [43]:
%%time
data = [
            {'sQuestion': 'the sum of 2 and a number is 1 . find the number .'
                , 'lEquations': ['37*8=x'],
             'iIndex': '122', 'lSolutions': ['4'], 'template': ''}
            # {'sQuestion': "Martin strolled to Lawrence's house. It is 12 miles from Martin's house to Lawrence's house. It took Martin 6 hours to get there. How fast did Martin go?"
            #     , 'lEquations': ['37*8=x'],
            #  'iIndex': '122', 'lSolutions': ['4'], 'template': ''}
        ]
res = dataset.preprocesss(data, use_gpu=False)

CPU times: total: 1.8 s
Wall time: 1.64 s


In [44]:
res_new = dataloader.build_batch_for_predict(res)

In [49]:
eqn = trainer.model.predict(res_new)[3][0]

[['-', '2.0', '1.0']]


C:\Users\goelm\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\torch\nn\modules\rnn.py:953: UserWarning: RNN module weights are not part of single contiguous chunk of memory. This means they need to be compacted at every call, possibly greatly increasing memory usage. To compact weights again call flatten_parameters(). (Triggered internally at  ..\aten\src\ATen\native\cudnn\RNN.cpp:968.)
  result = _VF.gru(input, batch_sizes, hx, self._flat_weights, self.bias,


In [3]:
def from_prefix_to_infix(expression):
    r"""convert prefix equation to infix equation

    Args:
        expression (list): prefix expression.

    Returns:
        (list): infix expression.
    """
    st = list()
    last_op = []
    priority = {"<BRG>": 0, "=": 1, "+": 2, "-": 2, "*": 3, "/": 3, "^": 4}
    expression.reverse()
    for symbol in expression:
        if symbol not in ['+', '-', '*', '/', '^', "=", "<BRG>"]:
            st.append([symbol])
        else:
            n_left = st.pop()
            n_right = st.pop()
            left_first = False
            right_first = False
            if len(n_left) > 1 and priority[last_op.pop()] < priority[symbol]:
                left_first = True
            if len(n_right) > 1 and priority[last_op.pop()] <= priority[symbol]:
                right_first = True
            if left_first:
                n_left = ['('] + n_left + [')']
            if right_first:
                n_right = ['('] + n_right + [')']
            st.append(n_left + [symbol] + n_right)
            last_op.append(symbol)
    res = st.pop()
    return res

def get_eqn(ques):
    data = [
            {'sQuestion': ques
                , 'lEquations': ['37*8=x'],
             'iIndex': '122', 'lSolutions': ['4'], 'template': ''}
            # {'sQuestion': "Martin strolled to Lawrence's house. It is 12 miles from Martin's house to Lawrence's house. It took Martin 6 hours to get there. How fast did Martin go?"
            #     , 'lEquations': ['37*8=x'],
            #  'iIndex': '122', 'lSolutions': ['4'], 'template': ''}
        ]
    res = dataset.preprocesss(data, use_gpu=False)
    res_new = dataloader.build_batch_for_predict(res)
    eqn = trainer.model.predict(res_new)[3][0]
    eqn = from_prefix_to_infix(eqn)
    eqn = 'x = '+' '.join(eqn)
    return eqn

In [6]:
import backend.graph as graph
from backend.comparator_tree import *

def get_ans_list(ques):
    eqn = get_eqn(ques)
    eqn_parsed = graph.eq_parser(eqn)
    tmp = []
    for j in range(len(tree)):
        if len(tree[j]) == 0:
            continue
        if compare_tree(eqn_parsed[0], tree[j][0]) and compare_tree(eqn_parsed[1], tree[j][1]):
            tmp.append(j)
    return tmp, eqn

In [ ]:

from flask import Flask, Response, request

app = Flask(__name__)


@app.route('/')
def hello():
    return "Hello World!"


@app.route('/solve', methods=['GET', 'POST'])
def eqn_solver_flask():
    if request.method == 'POST':
        print(request.json['question'])
        list, eqn = get_ans_list(request.json['question'])
        return {'list': list, 'eqn': eqn}
        # return question
    # print(ques)
    return {'list': 'wrong format'}


app.run(host='0.0.0.0', port=8000)

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on all addresses.
 * Running on http://192.168.50.170:8000/ (Press CTRL+C to quit)


the sum of 23 and a number is 1 . find the number .


C:\Users\goelm\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\torch\nn\modules\rnn.py:953: UserWarning: RNN module weights are not part of single contiguous chunk of memory. This means they need to be compacted at every call, possibly greatly increasing memory usage. To compact weights again call flatten_parameters(). (Triggered internally at  ..\aten\src\ATen\native\cudnn\RNN.cpp:968.)
  result = _VF.gru(input, batch_sizes, hx, self._flat_weights, self.bias,
192.168.50.170 - - [16/Apr/2023 02:01:50] "POST /solve HTTP/1.1" 200 -


[['-', '23.0', '1.0']]
( x  ) (  23.0 - 1.0 )
ram has 3 apples, shyam has 4 apples. How many they have in total?


192.168.50.170 - - [16/Apr/2023 02:02:25] "POST /solve HTTP/1.1" 200 -


[['+', '3.0', '4.0']]
( x  ) (  3.0 + 4.0 )
What is 4 times 5?


192.168.50.170 - - [16/Apr/2023 02:02:49] "POST /solve HTTP/1.1" 200 -


[['*', '4.0', '12.0']]
( x  ) (  4.0 * 12.0 )


In [ ]:
get_ans_list('the sum of 2 and a number is 1 . find the number .')